In [2]:
# %%capture
# !pip install git+https://github.com/biothings/biothings_explorer#egg=biothings_explorer

In [312]:
import os
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
from datetime import date
import datetime as dt
import pandas as pd
import sys
import csv
import numpy as np
import multiprocessing
import re
from collections import namedtuple
# For pausing
from time import sleep
import findspark
findspark.init()
import pyspark
os.environ['PYSPARK_SUBMIT_ARGS'] = """--name job_name --master local --conf spark.dynamicAllocation.enabled=true pyspark-shell"""
os.environ["JAVA_HOME"] = "/Library/Java/JavaVirtualMachines/jdk1.8.0_251.jdk/Contents/Home"
import pickle
import inspect
from itertools import repeat



In [189]:
import random

In [190]:
# adding Folder_2/subfolder to the system path
sys.path.insert(0, '/Volumes/TOSHIBA_EXT/ISB/clinical_trials/pymetamap-master')
from pymetamap import MetaMap


In [3]:
import psycopg2
import pandas.io.sql as sqlio
# from tabulate import tabulate

In [4]:
import json
import argparse
import lxml
import config
import pickle

# from Authentication import *
import requests
import json
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry


In [ ]:
url = "https://aact.ctti-clinicaltrials.org/pipe_files"
response = requests.get(url)

In [ ]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(response.text)
# print(soup.prettify())

In [ ]:

# https://aact.ctti-clinicaltrials.org/static/exported_files/monthly/20220301_pipe-delimited-export.zip
# https://aact.ctti-clinicaltrials.org/static/exported_files/monthly/20220801_pipe-delimited-export.zip
upload_dates = []
zip_files = []
links = []
body = soup.find_all('td', attrs={'class': 'file-archive'}) #Find all
for el in body:
    tags = el.find('a')
    try:
        if 'href' in tags.attrs:   # looking for href inside anchor tag    
            link = "https://aact.ctti-clinicaltrials.org" + tags.get('href')
            links.append(link)
            last_upload = link.split("/")[-1]
            zip_files.append(last_upload)
            date_upload = last_upload.split("_")[0]
            upload_dates.append(date_upload)    # appending link to list of links
    except:    # pass if list missing anchor tag or anchor tag does not has a href params 
        pass
    
print(upload_dates)

upload_dates = [dt.datetime.strptime(date, '%Y%m%d').date() for date in upload_dates] # convert all strings in list into datetime objects


In [ ]:
most_recent_date = max(upload_dates)
# print(most_recent_date)

today = dt.date.today()

# UNCOMMENT LINE BELOW WHEN IN PRODUCTION, THIS IS FOR TESTING PURPOSES
most_recent_date = min(upload_dates)

# fix this to run entire script should the current date be 
if most_recent_date < today:
#     date_file_df = pd.DataFrame(list(zip(upload_dates, zip_files, links)))
#     print(links[0])
    resp = urlopen(links[0])
else:
    print("Local instance of clinical trials data is already up to date.")



In [ ]:
pd.set_option('display.max_colwidth', 3000)
# date_file_df[2]

In [ ]:
os.getcwd()


In [333]:
# connect to DB and get the column names of the table
con = None
con = psycopg2.connect(database="aact")
con.rollback()
cursor = con.cursor()

con.autocommit = True # SQL statement is treated as a transaction and is automatically committed right after it is executed
# grab the conditions
sql = '''SELECT * FROM ctgov.conditions;'''
cursor.execute(sql)
column_names = [desc[0] for desc in cursor.description]
tuples = cursor.fetchall()
conditions_df = pd.DataFrame(tuples, columns=column_names)

#grab the browse_conditions
sql = '''SELECT * FROM ctgov.browse_conditions;'''
cursor.execute(sql)
column_names = [desc[0] for desc in cursor.description]
tuples = cursor.fetchall()
browse_conditions_df = pd.DataFrame(tuples, columns=column_names)

#grab the interventions
sql = '''SELECT * FROM ctgov.interventions;'''
cursor.execute(sql)
column_names = [desc[0] for desc in cursor.description]
tuples = cursor.fetchall()
interventions_df = pd.DataFrame(tuples, columns=column_names)

#grab the browse_interventions
sql = '''SELECT * FROM ctgov.browse_interventions;'''
cursor.execute(sql)
column_names = [desc[0] for desc in cursor.description]
tuples = cursor.fetchall()
browse_interventions_df = pd.DataFrame(tuples, columns=column_names)

con.close()


In [334]:
interventions_df = interventions_df.rename(columns={'id': 'int_id',
                                                    'nct_id': 'int_nctid',
                                                    'intervention_type': 'int_type',
                                                    'name': 'int_name',
                                                    'description': 'int_description'})
interventions_df = interventions_df.drop(columns=['int_id', 'int_description'])
conditions_df = conditions_df.rename(columns={'id': 'con_id',
                                              'nct_id': 'con_nctid',
                                              'name': 'con_name',
                                              'downcase_name': 'con_downcase_name'})
conditions_df = conditions_df.drop(columns=['con_id', 'con_name'])
browse_interventions_df = browse_interventions_df.rename(columns={'id': 'browseint_id',
                                                                  'nct_id': 'browseint_nctid',
                                                                  'mesh_term': 'browseint_meshterm',
                                                                  'downcase_mesh_term': 'browseint_meshterm_downcase',
                                                                  'mesh_type': 'browseint_meshtype'})
browse_interventions_df = browse_interventions_df.drop(columns=['browseint_id', 'browseint_meshterm'])
browse_conditions_df = browse_conditions_df.rename(columns={'id': 'browsecon_id',
                                                            'nct_id': 'browsecon_nctid',
                                                            'mesh_term': 'browsecon_meshterm',
                                                            'downcase_mesh_term': 'browsecon_meshterm_downcase',
                                                            'mesh_type': 'browsecon_meshtype'})
browse_conditions_df = browse_conditions_df.drop(columns=['browsecon_id', 'browsecon_meshterm'])


In [335]:
print(len(interventions_df['int_name'].unique()))
print(len(conditions_df['con_downcase_name'].unique()))
print(len(browse_conditions_df['browsecon_meshterm_downcase'].unique()))
print(len(browse_interventions_df['browseint_meshterm_downcase'].unique()))

350361
95250
4280
3744


In [336]:
df = pd.merge(conditions_df, interventions_df, left_on='con_nctid', right_on = 'int_nctid')
df.shape

(1256248, 5)

In [337]:
df_dedup = df.drop_duplicates(subset = ['con_downcase_name', 'int_name'],
                                      keep = 'first').reset_index(drop = True)
df_dedup.shape

(972685, 5)

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(df_dedup.head(1000))

In [ ]:
# df_dedup[['con_downcase_name', 'int_name']].to_csv('ClinTrials_KG_edges.csv', sep ='\t', index=False)


# notes on creating edge file

go to /proj/arivale/gglusman/KGs

once there look into wellness/source/wellness_kg_edges_v1.6.tsv

notice the first line: subject	predicate	object	relation	subject_name	object_name	category	N	Type_of_relationship	Strength_of_relationship	qualifiers	qualifier_value	Bonferroni_pval

subject, predicate, object are required (maybe some others, don’t remember)

…and they should be CURIEs

once we map concepts to biolink, mesh etc we’ll have curies

meanwhile, the content you have fits subject_name, object_name

the predicate is what we made up… let’s say ‘evaluated_for’

for short

we’ll have to find a suitable biolink predicate, or suggest a new one

so interventions would be subjects, and conditions would be objects

looking at the several first lines in your file, I see some that make sense, others that seem to suggest there are data problems

for example, “metastatic lung cancer	Pathways” -> Pathways doesn’t seem to be an intervention

‘placebo’ we’ll have to suppress

make sure to include the NCT id in a (non-required) column

it’s essentially part of the provenance

we’ll want to add multiple additional columns with things like FDA approval status, sample size, etc etc etc

look also into EHR/source/ehr_risk_kg_edges_2021_05_07.tsv

…and of course replace ‘edges’ -> ‘nodes’ for their cognate nodes files

wellness/source/wellness_kg_edges_v1.6.tsv -> wellness/source/wellness_kg_nodes_v1.6.tsv

drop "control"

‘control’ is clear, ‘control group’ too… but ‘Control Ostomy Barrier’ is likely not a ‘control’


# My own notes

Use pool/multiprocess to make faster
resolve multiple possible CURIEs per concept

In [338]:
# first get only relevant columns from DB
ct_extract = pd.DataFrame(df_dedup[['con_nctid', 'con_downcase_name', 'int_type', 'int_name']])
ct_extract = ct_extract.rename(columns={'con_nctid': 'nctid'})
# get CURIE column for nct_id column (https://bioregistry.io/registry/clinicaltrials)

ct_extract['nctid_curie'] = ct_extract['nctid']
ct_extract['nctid_curie'] = 'clinicaltrials:' + ct_extract['nctid'].astype(str)

In [339]:
ct_extract

,nctid,con_downcase_name,int_type,int_name,nctid_curie
0,NCT03254264,autism spectrum disorder,Behavioral,Early Start Denver Model,clinicaltrials:NCT03254264
1,NCT03254264,autism spectrum disorder,Behavioral,'as-usual' intervention,clinicaltrials:NCT03254264
2,NCT03254329,human milk nutrient reference values,Other,Assessment of human milk nutrient composition,clinicaltrials:NCT03254329
3,NCT03460652,adhd,Drug,KP415 oral capsule,clinicaltrials:NCT03460652
4,NCT03460899,diabetes mellitus with hypoglycemia,Other,Euglycaemic Clamp,clinicaltrials:NCT03460899
...,...,...,...,...,...
972680,NCT03261687,pelvic girdle pain,Other,water based exercise,clinicaltrials:NCT03261687
972681,NCT03261687,pelvic girdle pain,Other,land based exercise,clinicaltrials:NCT03261687
972682,NCT03261622,faecal incontinence,Device,Sacral nerve stimulation at different stimulat...,clinicaltrials:NCT03261622
972683,NCT03261622,fecal incontinence,Device,Sacral nerve stimulation at different stimulat...,clinicaltrials:NCT03261622


In [ ]:
# let's map all the conditions to BTE
# NOTE: we are attempting to match to BTE terms, but many of the conditions terms are not standardized and are misspelled
# First map as many terms as possible to BTE. Whatever is left over, will use other ontologies to create CURIEs


In [ ]:
# recommedation to first format the output as close to sample as possible

In [340]:
ct_final = pd.DataFrame(columns=['subject','predicate','object','subject_name','object_name','category'])
# no_col_names_df.columns = col_names_df.columns
# chosen predicate = "related_to", bc "associated_with" implies a statistical relationship 
ct_final['subject'] = 'condition:' + ct_extract['con_downcase_name'].astype(str)
ct_final['predicate'] = 'biolink:hypothesized_for'
ct_final['object'] = 'intervention:' + ct_extract['int_name'].astype(str)   # this will not all be RxNorm CURIEs since some interventions are not drugs
# ct_final['relation'] = 'RO:????'
ct_final.subject_name = ct_extract.con_downcase_name
ct_final.object_name = ct_extract.int_name
ct_final.category = 'biolink:Association'
ct_final['nctid'] = ct_extract['nctid']
ct_final['nctid_curie'] = ct_extract['nctid_curie']


In [341]:
ct_final[:10]

,subject,predicate,object,subject_name,object_name,category,nctid,nctid_curie
0,condition:autism spectrum disorder,biolink:hypothesized_for,intervention:Early Start Denver Model,autism spectrum disorder,Early Start Denver Model,biolink:Association,NCT03254264,clinicaltrials:NCT03254264
1,condition:autism spectrum disorder,biolink:hypothesized_for,intervention:'as-usual' intervention,autism spectrum disorder,'as-usual' intervention,biolink:Association,NCT03254264,clinicaltrials:NCT03254264
2,condition:human milk nutrient reference values,biolink:hypothesized_for,intervention:Assessment of human milk nutrient...,human milk nutrient reference values,Assessment of human milk nutrient composition,biolink:Association,NCT03254329,clinicaltrials:NCT03254329
3,condition:adhd,biolink:hypothesized_for,intervention:KP415 oral capsule,adhd,KP415 oral capsule,biolink:Association,NCT03460652,clinicaltrials:NCT03460652
4,condition:diabetes mellitus with hypoglycemia,biolink:hypothesized_for,intervention:Euglycaemic Clamp,diabetes mellitus with hypoglycemia,Euglycaemic Clamp,biolink:Association,NCT03460899,clinicaltrials:NCT03460899
5,condition:diabetes mellitus with hypoglycemia,biolink:hypothesized_for,intervention:Hyperinsulinaemic/Hypoglycaemic C...,diabetes mellitus with hypoglycemia,Hyperinsulinaemic/Hypoglycaemic Clamp,biolink:Association,NCT03460899,clinicaltrials:NCT03460899
6,"condition:diabetes mellitus, type 2",biolink:hypothesized_for,intervention:Euglycaemic Clamp,"diabetes mellitus, type 2",Euglycaemic Clamp,biolink:Association,NCT03460899,clinicaltrials:NCT03460899
7,"condition:diabetes mellitus, type 2",biolink:hypothesized_for,intervention:Hyperinsulinaemic/Hypoglycaemic C...,"diabetes mellitus, type 2",Hyperinsulinaemic/Hypoglycaemic Clamp,biolink:Association,NCT03460899,clinicaltrials:NCT03460899
8,condition:hypoglycemia,biolink:hypothesized_for,intervention:Euglycaemic Clamp,hypoglycemia,Euglycaemic Clamp,biolink:Association,NCT03460899,clinicaltrials:NCT03460899
9,condition:hypoglycemia,biolink:hypothesized_for,intervention:Hyperinsulinaemic/Hypoglycaemic C...,hypoglycemia,Hyperinsulinaemic/Hypoglycaemic Clamp,biolink:Association,NCT03460899,clinicaltrials:NCT03460899


In [ ]:
# interventions will have to be mapped to more than 1 vocabulary unlike conditions, which seems more easily mapped to SNOMED
# first look at the types of interventions (unique on int_type in ct_extract dataframe)

In [23]:
# get the directory of the the module
print(os.getcwd())
# Path
path = "/Volumes/TOSHIBA_EXT/ISB/clinical_trials/public_mm/bin"  
# Path of Start directory
start = os.getcwd()
print(os.path.relpath("/Volumes/TOSHIBA_EXT/ISB/clinical_trials/public_mm/bin", os.getcwd()))

/Users/Kamileh/Work/ISB/NCATS_BiomedicalTranslator/Projects/ClinicalTrials/ETL_Python
../../../../../../../../Volumes/TOSHIBA_EXT/ISB/clinical_trials/public_mm/bin


In [24]:
# For pausing
# from time import sleep

# NOTE: YOU MUST START the SKR/Medpost Part-of-Speech Tagger Server and the Word Sense Disambiguation (WSD) Server IN TERMINAL
# Setup UMLS Server
metamap_base_dir = '/Volumes/TOSHIBA_EXT/ISB/clinical_trials/public_mm/'
metamap_bin_dir = 'bin/metamap18'
metamap_pos_server_dir = 'bin/skrmedpostctl'
metamap_wsd_server_dir = 'bin/wsdserverctl'


# Start servers
os.system(metamap_base_dir + metamap_pos_server_dir + ' start') # Part of speech tagger
os.system(metamap_base_dir + metamap_wsd_server_dir + ' start') # Word sense disambiguation 

# Sleep a bit to give time for these servers to start up
sleep(60)


In [25]:
mm = MetaMap.get_instance(metamap_base_dir + metamap_bin_dir)

In [26]:
sents = ['myocardial infarction']  # this has to be 1 sentence
concepts,error = mm.extract_concepts(sents)
for concept in concepts:
    print(concept)
    print("\n")

ConceptMMI(index='USER', mm='MMI', score='13.22', preferred_name='Myocardial Infarction', cui='C0027051', semtypes='[dsyn]', trigger='["Infarction, Myocardial"-tx-1-"myocardial infarction"-noun-0]', location='TX', pos_info='4/21', tree_codes='C14.280.647.500;C14.907.585.500')


ConceptMMI(index='USER', mm='MMI', score='3.75', preferred_name='Electrocardiogram: myocardial infarction (finding)', cui='C0428953', semtypes='[lbtr]', trigger='["MYOCARDIAL INFARCTION"-tx-1-"myocardial infarction"-noun-0]', location='TX', pos_info='4/21', tree_codes='')


ConceptMMI(index='USER', mm='MMI', score='3.75', preferred_name='Myocardial Infarction ECG Assessment', cui='C3810814', semtypes='[diap]', trigger='["Myocardial Infarction"-tx-1-"myocardial infarction"-noun-0]', location='TX', pos_info='4/21', tree_codes='')


ConceptMMI(index='USER', mm='MMI', score='3.75', preferred_name='Myocardial Infarction, CTCAE', cui='C4552959', semtypes='[fndg]', trigger='["Myocardial infarction"-tx-1-"myocardial inf

In [28]:
diseases = list(set(list(ct_final['subject_name'])))
interventions = list(filter(None, ct_final['object_name'].unique()))

In [41]:
diseases[:2]

['pancreatic neoplasms', 'prodromal']

In [199]:
import re

non_ascii = "[^\x00-\x7F]"
pattern = re.compile(r"[^\x00-\x7F]")

def custom_make_translation(text):
    non_ascii_text = re.sub(pattern, ' ', text)
    return non_ascii_text


text = "split-virion, non-adjuvanted H1N1 vaccine of 15 μg"
translated = custom_make_translation(text)

In [30]:
translated

'split-virion, non-adjuvanted H1N1 vaccine of 15  g'

In [200]:
diseases_translated = dict((i, custom_make_translation(i)) for i in diseases)
interventions_translated = dict((i, custom_make_translation(i)) for i in interventions)

In [32]:
interventions_translated

{'Early Start Denver Model': 'Early Start Denver Model',
 "'as-usual' intervention": "'as-usual' intervention",
 'Assessment of human milk nutrient composition': 'Assessment of human milk nutrient composition',
 'KP415 oral capsule': 'KP415 oral capsule',
 'Euglycaemic Clamp': 'Euglycaemic Clamp',
 'Hyperinsulinaemic/Hypoglycaemic Clamp': 'Hyperinsulinaemic/Hypoglycaemic Clamp',
 'VX-659/TEZ/IVA': 'VX-659/TEZ/IVA',
 'TEZ/IVA': 'TEZ/IVA',
 'IVA': 'IVA',
 'Placebo': 'Placebo',
 'MT-6548': 'MT-6548',
 'Onasemnogene Abeparvovec-xioi': 'Onasemnogene Abeparvovec-xioi',
 'Rimegepant': 'Rimegepant',
 '3D printed model': '3D printed model',
 'Verbal Counselling': 'Verbal Counselling',
 'Asymmetric STN-DBS': 'Asymmetric STN-DBS',
 'Methacetin Breath Test': 'Methacetin Breath Test',
 'Emricasan': 'Emricasan',
 'Placebo oral capsule': 'Placebo oral capsule',
 'High fiber sugar free biscuit': 'High fiber sugar free biscuit',
 'CKD-501, Ketoconazole': 'CKD-501, Ketoconazole',
 'St. Johns Wort': 'St.

In [33]:
# test translation function on test list

# diseases_translated = dict((i, custom_make_translation(i)) for i in diseases[:20])
# interventions_translated = dict((i, custom_make_translation(i)) for i in interventions[:20])

In [87]:
def run_metamap(input_term, source_restriction):
    concepts_dict = dict()
    if all(x is None for x in source_restriction):
        try:
            concepts,error = mm.extract_concepts([input_term],
                                                 word_sense_disambiguation = True,
                                                 prune = 10,
                                                 composite_phrase = 1)
            concepts_dict[input_term] = concepts
        except:
            concepts_dict[input_term] = None 
    else:
        try:
            concepts,error = mm.extract_concepts([input_term],
                                                 word_sense_disambiguation = True,
                                                 restrict_to_sources=source_restriction,
                                                 prune = 10,
                                                 composite_phrase = 1)
            concepts_dict[input_term] = concepts
        except:
            concepts_dict[input_term] = None 
    return(concepts_dict)

In [33]:
avail_num_workers = multiprocessing.cpu_count() - 1  

In [88]:
# uncomment to run MetaMap on full dataset from Clinical Trials
# number_of_processes = 4
# metamapped_diseases = multiprocessing.Pool(multiprocessing.cpu_count() - 1).starmap(run_metamap,
#                                                                         zip(list(diseases_translated.values()), [['SNOMEDCT_US', 'SNOMEDCT_VET', 'ICD10CM', 'ICD10CM', 'ICD10PCS', 'ICD9CM', 'ICD9CM']]*len(list(diseases_translated.values()))))
# metamapped_interventions = multiprocessing.Pool(multiprocessing.cpu_count() - 1).starmap(run_metamap, 
#                                                                              zip(list(interventions_translated.values()),
#                                                                                  [[None]]*len(list(interventions_translated.values()))))
                                                                             
# Stop MetaMap servers
os.system(metamap_base_dir + metamap_pos_server_dir + ' stop') # Part of speech tagger
os.system(metamap_base_dir + metamap_wsd_server_dir + ' stop') # Word sense disambiguation 

In [34]:
metamapped_diseases[0]
type(metamapped_diseases[0])
test_dict = metamapped_diseases[:2]
test_dict

NameError: name 'metamapped_diseases' is not defined

In [ ]:
# # CAREFUL: UNCOMMENTING WILL OVERWRITE METAMAPPED DISEASES AND INTERVENTIONS FILES, WHICH TAKE FOREVER TO GENERATE!

# # write mapped diseases and interventions to pickle file
# with open('metamapped_diseases.txt', 'wb') as file:
#     pickle.dump(metamapped_diseases, file)
# with open('metamapped_interventions.txt', 'wb') as file:
#     pickle.dump(metamapped_interventions, file)

# # write mapped diseases and interventions to human readable file
# with open('readable_metamapped_diseases.txt', 'w') as file:
#     file.write(json.dumps(metamapped_diseases, indent=4))
# with open('readable_metamapped_interventions.txt', 'w') as file:
#     file.write(json.dumps(metamapped_interventions, indent=4))

In [35]:
with open('metamapped_diseases.txt', 'rb') as f:
    metamapped_diseases = pickle.load(f)
    
with open('metamapped_interventions.txt', 'rb') as f:
    metamapped_interventions = pickle.load(f)

In [107]:
# # takes way too long

# master_disease_dict = dict()
# master_intervention_dict = dict()

for disease in metamapped_diseases:
    for key, val in disease.items():
        for concept in val:
            concept = concept._asdict()
#             row = [key]
            from_metamap = []
            original_concept = (list(diseases_translated.keys())[list(diseases_translated.values()).index(key)])
#             row.append(original_concept)
#             row.extend([concept.get(k) for k in ['preferred_name', 'cui', 'score', 'semtypes']])
            from_metamap.extend([concept.get(k) for k in ['preferred_name', 'cui', 'score', 'semtypes']])
#             print(row)
            master_disease_dict[original_concept] = from_metamap
            break

# for intervention in metamapped_interventions:
#     for key, val in intervention.items():
#         for concept in val:
#             concept = concept._asdict()
# #             row = [key]
#             from_metamap = []
#             original_concept = (list(interventions_translated.keys())[list(interventions_translated.values()).index(key)])
# #             row.append(original_concept)
# #             row.extend([concept.get(k) for k in ['preferred_name', 'cui', 'score', 'semtypes']])
#             from_metamap.extend([concept.get(k) for k in ['preferred_name', 'cui', 'score', 'semtypes']])
# #             print(row)
#             master_intervention_dict[original_concept] = from_metamap
#             break

KeyboardInterrupt: 

In [77]:
# # write the mapping results to text files for evaluation 

# with open("metamapped_diseases_qual_control.csv", "w") as file:
#     csv.excel.delimiter=','
#     writer = csv.writer(file, dialect=csv.excel)
#     writer.writerow(['original_ct_value', 'metamap_input', 'metamap_input', 'metamap_cui', 'metamap_score', 'metamap_semtypes'])
#     for disease in metamapped_diseases[:100]:
#         for key in disease:
#             for concept in disease[key]:
#                 concept = concept._asdict()
#                 row = [key]
#                 original_disease = (list(diseases_translated.keys())[list(diseases_translated.values()).index(key)])
#                 row.append(original_disease)
#                 row.extend([concept.get(k) for k in ['preferred_name', 'cui', 'score', 'semtypes']])
#                 writer.writerow(row)
#                 print(row)
#                 break
        


['infertile high-risk patients with polycystic ovaries', 'infertile high-risk patients with polycystic ovaries', 'Polycystic Ovary Syndrome', 'C0032460', '22.36', '[dsyn]']
['anaesthetic induction', 'anaesthetic induction', 'Anesthetics', 'C0002932', '19.25', '[phsu]']
['riss stage ii plasma cell myeloma', 'riss stage ii plasma cell myeloma', 'Multiple Myeloma', 'C0026764', '16.42', '[neop]']
['metabolic abnormality', 'metabolic abnormality', 'Abnormal', 'C0205161', '3.55', '[fndg]']
['chalazion unspecified eye, unspecified eyelid', 'chalazion unspecified eye, unspecified eyelid', 'Eyelid structure', 'C0015426', '16.30', '[bpoc]']
['refractory transformed marginal zone lymphoma to diffuse large b-cell lymphoma', 'refractory transformed marginal zone lymphoma to diffuse large b-cell lymphoma', 'Diffuse Large B-Cell Lymphoma', 'C0079744', '22.58', '[neop]']
['internet based wellness program', 'internet based wellness program', 'Alkalies', 'C0002055', '7.13', '[inch]']
['infant colics', '

['enzyme disorder', 'enzyme disorder', 'Enzymopathy', 'C0520572', '3.75', '[dsyn]']
['atypical burkitt/burkitt-like lymphoma', 'atypical burkitt/burkitt-like lymphoma', 'Burkitt-like lymphoma', 'C1368771', '3.88', '[neop]']
['adult fibrosarcoma', 'adult fibrosarcoma', 'Fibrosarcoma', 'C0016057', '19.40', '[neop]']
['burkitts lymphoma', 'burkitts lymphoma', 'Burkitt Lymphoma', 'C0006413', '22.68', '[neop]']
['closed-loop glucose control', 'closed-loop glucose control', 'Glucose', 'C0017725', '16.22', '[bacs,orch,phsu]']
['skeletal muscle physiology', 'skeletal muscle physiology', 'Skeletal muscle structure', 'C0242692', '12.96', '[tisu]']
['neoplasms, neck', 'neoplasms, neck', 'Neck', 'C0027530', '8.34', '[blor]']
['mind-body-medicine', 'mind-body-medicine', 'Body structure', 'C1268086', '3.61', '[anst]']
['evaluating impact of pacv on vaccine hesitancy among parents of newborns', 'evaluating impact of pacv on vaccine hesitancy among parents of newborns', 'parent', 'C0030551', '19.19', 

In [133]:
def get_mapped_ct_data(metamapped_concept, translation_dict):
    for key, val in metamapped_concept.items():
        for concept in val:
            concept = concept._asdict()
            row = [key]
            original_concept = (list(translation_dict.keys())[list(translation_dict.values()).index(key)])
            row.append(original_concept)
            row.extend([concept.get(k) for k in ['preferred_name', 'cui', 'score', 'semtypes']])
            return row # we only want the first mapped concept
    
# for testing using subsamples            
mm_diseases = metamapped_diseases[:100]
mm_interventions = metamapped_interventions[:100]


with multiprocessing.Pool(multiprocessing.cpu_count() - 1) as pool:
    final_metamapped_diseases = pool.starmap(get_mapped_ct_data, zip(metamapped_diseases, repeat(diseases_translated)))   
    final_metamapped_interventions = pool.starmap(get_mapped_ct_data, zip(metamapped_interventions, repeat(interventions_translated)))

master_disease_dict = {disease[0]:disease[1:] for disease in final_metamapped_diseases if disease}  
master_intervention_dict = {intervention[0]:intervention[1:] for intervention in final_metamapped_interventions if intervention}    


    
#     print(final_metamapped_interventions)
    
#     pool.map(write_mapped_ct_data, final_metamapped_diseases)
#     pool.map(write_mapped_ct_data, final_metamapped_interventions)

    


In [ ]:
master_disease_dict = {disease[0]:disease[1:] for disease in final_metamapped_diseases if disease}  
master_intervention_dict = {intervention[0]:intervention[1:] for intervention in final_metamapped_interventions if intervention}    


In [136]:
# # CAREFUL: UNCOMMENTING WILL OVERWRITE MASTER DISEASES DICT AND MASTER INTERVENTIONS DICT FILES, WHICH TAKE FOREVER TO GENERATE!

# # write master dict of mapped diseases and interventions to pickle file
## with open('master_disease_dict.txt', 'wb') as file:
##     pickle.dump(master_disease_dict, file)
## with open('master_intervention_dict.txt', 'wb') as file:
##     pickle.dump(master_intervention_dict, file)

# write master dict of mapped diseases and interventions to human readable file
## with open('readable_master_disease_dict.txt', 'w') as file:
##     file.write(json.dumps(master_disease_dict, indent=4))
## with open('readable_master_intervention_dict.txt', 'w') as file:
##     file.write(json.dumps(master_intervention_dict, indent=4))

In [137]:
with open('master_disease_dict.txt', 'rb') as f:
    master_disease_dict = pickle.load(f)
    
with open('master_intervention_dict.txt', 'rb') as f:
    master_intervention_dict = pickle.load(f)

In [143]:
test_master_dis_dict = dict(random.sample(master_disease_dict.items(), 100))
test_master_int_dict = dict(random.sample(master_intervention_dict.items(), 100))

In [145]:
test_master_dis_dict

{'pregnancy-induced diabetes': ['pregnancy-induced diabetes',
  'Gestational Diabetes',
  'C0085207',
  '16.42',
  '[dsyn]'],
 'alcohol exposed pregnancy': ['alcohol exposed pregnancy',
  'Ethanol',
  'C0001962',
  '13.08',
  '[orch,phsu]'],
 'ad hoc percutaneous coronary intervention': ['ad hoc percutaneous coronary intervention',
  'Percutaneous Coronary Intervention',
  'C1532338',
  '16.30',
  '[topp]'],
 'refractory malignant glioma': ['refractory malignant glioma',
  'Malignant Glioma',
  'C0555198',
  '19.45',
  '[neop]'],
 'arthroplasty, knee replacement': ['arthroplasty, knee replacement',
  'Knee Replacement Arthroplasty (procedure)',
  'C0086511',
  '17.80',
  '[topp]'],
 'proprotein convertase subtilisin/kexin type 9 inhibitor': ['proprotein convertase subtilisin/kexin type 9 inhibitor',
  'Proprotein convertase subtilisin/kexin type 9 inhibitor (disposition)',
  'C4522007',
  '3.94',
  '[qlco]'],
 'stage iii/iv melanoma': ['stage iii/iv melanoma',
  'melanoma',
  'C0025202

In [170]:
for k, v in test_master_dis_dict.items():
#     l = [k,v[1]]
#     print(l)
    
    orig_disease = k.lower()
    orig_disease = orig_disease.strip()
#     print(str(v[1]))
    mapped_disease = str(v[1])
    mapped_disease = mapped_disease.lower()
    mapped_disease = mapped_disease.strip()
    if orig_disease == mapped_disease:  
        l = [orig_disease, mapped_disease]
        print(l)
        

['single right ventricle', 'single right ventricle']
['lynch syndrome', 'lynch syndrome']
['intravenous anesthetic agent overdose', 'intravenous anesthetic agent overdose']
['enzyme inhibitors', 'enzyme inhibitors']
['typhoid vaccination', 'typhoid vaccination']
['avocado', 'avocado']
['isthmic spondylolisthesis', 'isthmic spondylolisthesis']
['extensively drug-resistant tuberculosis', 'extensively drug-resistant tuberculosis']
['altitude', 'altitude']
['antineoplastic adverse reaction', 'antineoplastic adverse reaction']
['deviated nasal septum', 'deviated nasal septum']
['osteoarthritis of acromioclavicular joint', 'osteoarthritis of acromioclavicular joint']
['meibomianitis', 'meibomianitis']


In [178]:
baseline_final_disease_dict = dict()
baseline_final_intervention_dict = dict()

for k, v in master_disease_dict.items():
    orig_disease = k.lower()
    orig_disease = orig_disease.strip()
    mapped_disease = str(v[1])
    mapped_disease = mapped_disease.lower()
    mapped_disease = mapped_disease.strip()
    if orig_disease == mapped_disease:
        baseline_final_disease_dict[k] = v[1:] 
        
for k, v in master_intervention_dict.items():
    orig_int = k.lower()
    orig_int = orig_int.strip()
    mapped_int = str(v[1])
    mapped_int = mapped_int.lower()
    mapped_int = mapped_int.strip()
    if orig_int == mapped_int:
        baseline_final_intervention_dict[k] = v[1:] 

        

In [201]:
baseline_final_intervention_dict

{'Rimegepant': ['Rimegepant', 'C4519126', '3.68', '[orch,phsu]'],
 'Emricasan': ['Emricasan', 'C2699871', '3.68', '[orch,phsu]'],
 'EMLA': ['EMLA', 'C0059079', '3.68', '[orch,phsu]'],
 'metoprolol': ['Metoprolol', 'C0025859', '19.45', '[orch,phsu]'],
 'eye patch': ['Eye patch', 'C0776981', '3.75', '[medd]'],
 'niacin': ['Niacin', 'C0027996', '16.30', '[orch,phsu,vita]'],
 'Pembrolizumab': ['pembrolizumab', 'C3658706', '13.14', '[aapp,imft,phsu]'],
 'Epacadostat': ['epacadostat', 'C4086265', '13.14', '[orch,phsu]'],
 'Magnetic Resonance Imaging': ['Magnetic Resonance Imaging',
  'C0024485',
  '16.42',
  '[diap]'],
 'Lasmiditan': ['Lasmiditan', 'C4519199', '3.68', '[orch,phsu]'],
 'Moxifloxacin': ['moxifloxacin', 'C0536495', '22.61', '[antb,orch]'],
 'Coffee': ['Coffee', 'C0009237', '13.14', '[food]'],
 'Decaffeinated coffee': ['decaffeinated coffee',
  'C0681565',
  '3.75',
  '[food]'],
 'Atomoxetine': ['atomoxetine', 'C0076823', '3.68', '[orch,phsu]'],
 'Polihexanide': ['polihexanide',

In [347]:
# create CURIEs from values of baseline lists
disease_curie_dict = dict()
intervention_curie_dict = dict()

for k, v in baseline_final_disease_dict.items():
    disease_curie_dict[k] = "UMLS:{}".format(v[1])
    
for k, v in baseline_final_intervention_dict.items():
    intervention_curie_dict[k] = "UMLS:{}".format(v[1])
    
    
disease_curie_dict

{'heart valves': 'UMLS:C0018826',
 'primary enuresis': 'UMLS:C0232847',
 'coronary occlusion': 'UMLS:C0151814',
 'immune tolerance': 'UMLS:C0020963',
 'dwarfism': 'UMLS:C0013336',
 'b-cell lymphomas': 'UMLS:C0079731',
 'alveolar bone grafting': 'UMLS:C0407708',
 'failure of osseointegration of dental implant': 'UMLS:C2711774',
 'vibrio infections': 'UMLS:C0042636',
 'ocular hypertension': 'UMLS:C0028840',
 'psychoeducation': 'UMLS:C0871175',
 'pain score': 'UMLS:C0582148',
 'premature ejaculation': 'UMLS:C0033038',
 'genital herpes simplex type 2': 'UMLS:C1827915',
 'displaced tooth': 'UMLS:C0155940',
 'cicatrix': 'UMLS:C2004491',
 'patient participation': 'UMLS:C0030699',
 'dabigatran': 'UMLS:C2348066',
 'myoclonus': 'UMLS:C0027066',
 'aspergilloma': 'UMLS:C0276651',
 'adult syndrome': 'UMLS:C1863204',
 'magnetic resonance spectroscopy': 'UMLS:C0024487',
 'back injuries': 'UMLS:C0004601',
 'clear cell sarcoma of soft tissue': 'UMLS:C0206651',
 'adhesive capsulitis of unspecified shoul

In [348]:
ct_final[:2]

,subject,predicate,object,subject_name,object_name,category,nctid,nctid_curie
0,condition:autism spectrum disorder,biolink:hypothesized_for,intervention:Early Start Denver Model,autism spectrum disorder,Early Start Denver Model,biolink:Association,NCT03254264,clinicaltrials:NCT03254264
1,condition:autism spectrum disorder,biolink:hypothesized_for,intervention:'as-usual' intervention,autism spectrum disorder,'as-usual' intervention,biolink:Association,NCT03254264,clinicaltrials:NCT03254264


In [349]:
ct_final_mapped = ct_final.copy()
ct_final_mapped['subject'] = ct_final['subject_name'].map(disease_curie_dict)
ct_final_mapped['object'] = ct_final['object_name'].map(intervention_curie_dict)

In [350]:
ct_final_mapped[:20]

,subject,predicate,object,subject_name,object_name,category,nctid,nctid_curie
0,NaN,biolink:hypothesized_for,NaN,autism spectrum disorder,Early Start Denver Model,biolink:Association,NCT03254264,clinicaltrials:NCT03254264
1,NaN,biolink:hypothesized_for,NaN,autism spectrum disorder,'as-usual' intervention,biolink:Association,NCT03254264,clinicaltrials:NCT03254264
2,NaN,biolink:hypothesized_for,NaN,human milk nutrient reference values,Assessment of human milk nutrient composition,biolink:Association,NCT03254329,clinicaltrials:NCT03254329
3,NaN,biolink:hypothesized_for,NaN,adhd,KP415 oral capsule,biolink:Association,NCT03460652,clinicaltrials:NCT03460652
4,NaN,biolink:hypothesized_for,NaN,diabetes mellitus with hypoglycemia,Euglycaemic Clamp,biolink:Association,NCT03460899,clinicaltrials:NCT03460899
5,NaN,biolink:hypothesized_for,NaN,diabetes mellitus with hypoglycemia,Hyperinsulinaemic/Hypoglycaemic Clamp,biolink:Association,NCT03460899,clinicaltrials:NCT03460899
6,NaN,biolink:hypothesized_for,NaN,"diabetes mellitus, type 2",Euglycaemic Clamp,biolink:Association,NCT03460899,clinicaltrials:NCT03460899
7,NaN,biolink:hypothesized_for,NaN,"diabetes mellitus, type 2",Hyperinsulinaemic/Hypoglycaemic Clamp,biolink:Association,NCT03460899,clinicaltrials:NCT03460899
8,UMLS:C0020615,biolink:hypothesized_for,NaN,hypoglycemia,Euglycaemic Clamp,biolink:Association,NCT03460899,clinicaltrials:NCT03460899
9,UMLS:C0020615,biolink:hypothesized_for,NaN,hypoglycemia,Hyperinsulinaemic/Hypoglycaemic Clamp,biolink:Association,NCT03460899,clinicaltrials:NCT03460899


In [351]:
ct_final_mapped = ct_final_mapped.dropna(subset=['subject','object'])
ct_final_mapped[:30]

,subject,predicate,object,subject_name,object_name,category,nctid,nctid_curie
45,UMLS:C0008059,biolink:hypothesized_for,UMLS:C0776981,child,eye patch,biolink:Association,NCT02590744,clinicaltrials:NCT02590744
49,UMLS:C0085631,biolink:hypothesized_for,UMLS:C0776981,agitation,eye patch,biolink:Association,NCT02590744,clinicaltrials:NCT02590744
90,UMLS:C0006644,biolink:hypothesized_for,UMLS:C0009237,caffeine,Coffee,biolink:Association,NCT01330680,clinicaltrials:NCT01330680
91,UMLS:C0006644,biolink:hypothesized_for,UMLS:C0681565,caffeine,Decaffeinated coffee,biolink:Association,NCT01330680,clinicaltrials:NCT01330680
95,UMLS:C0027339,biolink:hypothesized_for,UMLS:C0376892,nail diseases,Polihexanide,biolink:Association,NCT01330706,clinicaltrials:NCT01330706
111,UMLS:C0038013,biolink:hypothesized_for,UMLS:C1608841,ankylosing spondylitis,Ustekinumab,biolink:Association,NCT01330901,clinicaltrials:NCT01330901
126,UMLS:C0029463,biolink:hypothesized_for,UMLS:C0072980,osteosarcoma,sirolimus,biolink:Association,NCT01331135,clinicaltrials:NCT01331135
127,UMLS:C0004114,biolink:hypothesized_for,UMLS:C0072980,astrocytoma,sirolimus,biolink:Association,NCT01331135,clinicaltrials:NCT01331135
129,UMLS:C0014474,biolink:hypothesized_for,UMLS:C0072980,ependymoma,sirolimus,biolink:Association,NCT01331135,clinicaltrials:NCT01331135
130,UMLS:C0205851,biolink:hypothesized_for,UMLS:C0072980,germ cell tumor,sirolimus,biolink:Association,NCT01331135,clinicaltrials:NCT01331135


In [352]:
# make dataframe for nodes file derived from edges dataframe (ct_final_mapped)

ct_nodes_subjects = ct_final_mapped[['subject', 'subject_name']]
ct_nodes_subjects.columns = ['id', 'name']
ct_nodes_subjects = ct_nodes_subjects.assign(category='biolink:DiseaseOrPhenotypicFeature')

ct_nodes_objects = ct_final_mapped[['object', 'object_name']]
ct_nodes_objects.columns = ['id', 'name']
ct_nodes_objects = ct_nodes_objects.assign(category='biolink:Treatment')

ct_nodes = pd.concat([ct_nodes_subjects, ct_nodes_objects], ignore_index=True, axis=0)
ct_nodes = ct_nodes.drop_duplicates('id')




In [354]:
ct_nodes

,id,name,category
0,UMLS:C0008059,child,biolink:DiseaseOrPhenotypicFeature
1,UMLS:C0085631,agitation,biolink:DiseaseOrPhenotypicFeature
2,UMLS:C0006644,caffeine,biolink:DiseaseOrPhenotypicFeature
4,UMLS:C0027339,nail diseases,biolink:DiseaseOrPhenotypicFeature
5,UMLS:C0038013,ankylosing spondylitis,biolink:DiseaseOrPhenotypicFeature
...,...,...,...
111933,UMLS:C3209471,Azithromycin Oral Liquid Product,biolink:Treatment
111934,UMLS:C1252568,Nitazoxanide Oral Suspension,biolink:Treatment
111946,UMLS:C0039195,cytotoxic t-lymphocytes,biolink:Treatment
111963,UMLS:C0353942,Bran,biolink:Treatment


In [353]:
ct_final_mapped.to_csv('ClinTrials_KG_edges_v01.csv', sep ='\t', index=False)
ct_nodes.to_csv('ClinTrials_KG_nodes_v01.csv', sep ='\t', index=False)



In [ ]:
def get_mapped_ct_data(metamapped_concept, translation_dict):
    for key, val in metamapped_concept.items():
        for concept in val:
            concept = concept._asdict()
            row = [key]
            original_concept = (list(translation_dict.keys())[list(translation_dict.values()).index(key)])
            row.append(original_concept)
            row.extend([concept.get(k) for k in ['preferred_name', 'cui', 'score', 'semtypes']])
            return row
    
# for testing using subsamples            
mm_diseases = metamapped_diseases[:100]
mm_interventions = metamapped_interventions[:100]

with multiprocessing.Pool(multiprocessing.cpu_count() - 1) as pool:
    final_metamapped_diseases = pool.starmap(get_mapped_ct_data, zip(mm_diseases, repeat(diseases_translated)))   
    final_metamapped_interventions = pool.starmap(get_mapped_ct_data, zip(mm_interventions, repeat(interventions_translated)))
    
    
    
    

In [139]:
from pyspark.sql import Row
# spark - sparkSession
# sc = spark.sparkContext

# Load a text file and convert each line to a Row.
ct_data = sc.textFile("metamapped_diseases_qual_control.csv")
#Split on delimiters
ct_items = ct_data.map(lambda l: l.split(","))
#Convert to Row
ct_data_struct = ct_items.map(lambda p: Row(original_ct_value=int(p[0]),
                                        metamap_input=p[1],
                                        metamap_cui=p[2],
                                        metamap_score=p[3],
                                        metamap_semtypes=[4]))
for i in ct_data_struct.take(5): print(i)
#convert the RDD to DataFrame

ct_df = spark.createDataFrame(ct_data_struct)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 1 times, most recent failure: Lost task 0.0 in stage 0.0 (TID 0) (192.168.0.119 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/opt/apache-spark/libexec/python/lib/pyspark.zip/pyspark/worker.py", line 619, in main
    process()
  File "/usr/local/opt/apache-spark/libexec/python/lib/pyspark.zip/pyspark/worker.py", line 611, in process
    serializer.dump_stream(out_iter, outfile)
  File "/usr/local/opt/apache-spark/libexec/python/lib/pyspark.zip/pyspark/serializers.py", line 259, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/usr/local/opt/apache-spark/libexec/python/pyspark/rdd.py", line 1562, in takeUpToNumLeft
    yield next(iterator)
  File "/usr/local/opt/apache-spark/libexec/python/lib/pyspark.zip/py

In [143]:
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import StringType, ArrayType

schema = StructType(
    Array(
        StructField("original_ct_value", StringType,true),
        StructField("metamap_input", StringType,true),
        StructField("metamap_output_preferred_name", StringType,true),
        StructField("cui", StringType,true),
        StructField("score", StringType,true),
        StructField("semtypes", StringType,true),
        
))

test = spark.read.csv(
    "metamapped_diseases_qual_control.csv", 
    header=True, 
    mode="DROPMALFORMED", 
    schema=schema
)

NameError: name 'array' is not defined

In [136]:
from pyspark.sql import Row
# spark - sparkSession
sc = spark.sparkContext

# Load a text file and convert each line to a Row.
orders = sc.textFile("/practicedata/orders")
#Split on delimiters
parts = orders.map(lambda l: l.split(","))
#Convert to Row
orders_struct = parts.map(lambda p: Row(order_id=int(p[0]), order_date=p[1], customer_id=p[2], order_status=p[3]))
for i in orders_struct.take(5): print(i)
#convert the RDD to DataFrame

orders_df = spark.createDataFrame(orders_struct)

AttributeError: 'NoneType' object has no attribute 'sc'

In [137]:
# import pyspark
from pyspark.sql import Row

spark = SparkSession.builder.master("local[2]") \
                    .appName('clinical_trials_etl') \
                    .getOrCreate()

# # test = spark.createDataFrame(Row(**x) for x in metamapped_diseases[:20]).show(truncate=False)
# for disease in metamapped_diseases:
#         for key in disease:
#             for concept in disease[key]:
#                 concept = concept._asdict()
#                 row = [key]
#                 row.extend([concept.get(k) for k in ['preferred_name', 'cui', 'score', 'semtypes']])
#                 writer.writerow(row)


In [29]:
metamapped_diseases[:2]

[{'infertile high-risk patients with polycystic ovaries': [ConceptMMI(index='USER', mm='MMI', score='22.36', preferred_name='Polycystic Ovary Syndrome', cui='C0032460', semtypes='[dsyn]', trigger='["Polycystic ovary syndrome"-tx-1-"polycystic ovaries"-noun-0]', location='TX', pos_info='38/18', tree_codes='C04.182.612.765;C13.351.500.056.630.580.765;C19.391.630.580.765'),
   ConceptMMI(index='USER', mm='MMI', score='12.88', preferred_name='Infertility', cui='C0021359', semtypes='[patf]', trigger='["Infertility"-tx-1-"infertile"-adj-0]', location='TX', pos_info='4/9', tree_codes='C12.294.365;C13.351.500.365'),
   ConceptMMI(index='USER', mm='MMI', score='6.70', preferred_name='Patients', cui='C0030705', semtypes='[podg]', trigger='["Patient"-tx-1-"patients"-noun-0]', location='TX', pos_info='24/8', tree_codes='M01.643'),
   ConceptMMI(index='USER', mm='MMI', score='3.57', preferred_name='High risk', cui='C4319571', semtypes='[fndg]', trigger='["High risk"-tx-1-"high-risk"-noun-0]', locat

In [ ]:
with open("clinical_trials_kg_edges_v1.csv", "w") as file:
    csv.excel.delimiter='\t'
    writer = csv.writer(file, dialect=csv.excel)
#     writer.writerow(['original_ct_value', 'metamap_input', 'metamap_input', 'metamap_cui', 'metamap_score', 'metamap_semtypes'])
    writer.writerow(['original_ct_value', 'metamap_input', 'metamap_input', 'metamap_cui', 'metamap_score', 'metamap_semtypes'])

    for disease in metamapped_diseases[:100]:
        for key in disease:
            for concept in disease[key]:
                concept = concept._asdict()
                row = [key]
                original_disease = (list(diseases_translated.keys())[list(diseases_translated.values()).index(key)])
                row.append(original_disease)
                row.extend([concept.get(k) for k in ['preferred_name', 'cui', 'score', 'semtypes']])
                writer.writerow(row)
                print(row)
                break

In [25]:
from pyspark.sql import Row
spark.createDataFrame(Row(**x) for x in mylist).show(truncate=False)

[{'infertile high-risk patients with polycystic ovaries': [ConceptMMI(index='USER', mm='MMI', score='22.36', preferred_name='Polycystic Ovary Syndrome', cui='C0032460', semtypes='[dsyn]', trigger='["Polycystic ovary syndrome"-tx-1-"polycystic ovaries"-noun-0]', location='TX', pos_info='38/18', tree_codes='C04.182.612.765;C13.351.500.056.630.580.765;C19.391.630.580.765'),
   ConceptMMI(index='USER', mm='MMI', score='12.88', preferred_name='Infertility', cui='C0021359', semtypes='[patf]', trigger='["Infertility"-tx-1-"infertile"-adj-0]', location='TX', pos_info='4/9', tree_codes='C12.294.365;C13.351.500.365'),
   ConceptMMI(index='USER', mm='MMI', score='6.70', preferred_name='Patients', cui='C0030705', semtypes='[podg]', trigger='["Patient"-tx-1-"patients"-noun-0]', location='TX', pos_info='24/8', tree_codes='M01.643'),
   ConceptMMI(index='USER', mm='MMI', score='3.57', preferred_name='High risk', cui='C4319571', semtypes='[fndg]', trigger='["High risk"-tx-1-"high-risk"-noun-0]', locat

In [11]:


# pip show pyspark


# import pyspark
# from pyspark.sql import SparkSession
# spark = SparkSession.builder.master("local[1]").appName("SparkByExamples.com").getOrCreate()


In [135]:
spark.stop()

In [21]:


# spark = SparkSession.builder.appName("test").config(
#     "spark.driver.extraJavaOptions",
#     "--add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED",
# ).getOrCreate()

In [ ]:
# link nct ids of studies that have the same disease and intervention pair---multiple studies may have studies the same intervention
# restrict returned CUIs of metamapped concepts???

In [13]:
user = os.environ['USER']

In [14]:
user

'Kamileh'

In [23]:
import subprocess
version = subprocess.check_output(['java', '-version'], stderr=subprocess.STDOUT)
version

b'java version "1.8.0_251"\nJava(TM) SE Runtime Environment (build 1.8.0_251-b08)\nJava HotSpot(TM) 64-Bit Server VM (build 25.251-b08, mixed mode)\n'

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
mapped